<a href="https://colab.research.google.com/github/gpt2ent/gpt2colab-js/blob/master/GPT2_with_Javascript_interface_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This is proof of concept that GPT-2 can be run from colab with Javascript interface
**Q: How to do?**

A: 
1. Runtime -> Change runtime type -> Hardware accelerator -> GPU
2. Runtime -> Reset all runtimes
3. Runtime -> Run all

In [0]:
%tensorflow_version 1.x
!pip install gpt-2-simple

import gpt_2_simple as gpt2
import os
import requests

model_name = "1558M"
if not os.path.isdir(os.path.join("models", model_name)):
	  print(f"Downloading {model_name} model...")
	  gpt2.download_gpt2(model_name=model_name)
   
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, model_name=model_name)

import google.colab.output

def ai_generate(prefix, temp, top_k, length):
    temp = float(temp)
    top_k = int(top_k)
    length = int(length)
    result = gpt2.generate(sess, model_name=model_name, prefix=prefix, temperature=temp,
                         top_k=top_k, length=length, include_prefix=False, return_as_list=True)[0]
    if result.startswith(prefix):
        result = result[len(prefix):]
    return result

google.colab.output.register_callback('ai_generate', ai_generate)
print('Done')

In [0]:
from IPython.display import HTML

input_form = """
<div style="background-color:gainsboro; border:solid black; width:800px; padding:20px; color: black;">
<textarea id="main_textarea" cols="95" rows="30"></textarea><br>
Temperature: <input type="number" min="0.00" max="999.99" step="0.01" id="temp" value="0.70"><br>
TOP_K: <input type="number" min="0" max="9999" id="top_k" value="40"><br>
Generate how much: <input type="number" id="length" min="1" max="1023" value="10"><br>
<button onclick="generate()">Let AI continue</button>
<p style="color: red;" id="error-msg"></p>
</div>
"""

javascript = """
<script type="text/Javascript">
    function desanitize(text) {
        return text.slice(1,-1).replace(/\\\\n/g, "\\n").replace(/\\'/g, "'");
    };

    function add_text(text) {
        var deftext = document.getElementById('main_textarea').value;
        document.getElementById('main_textarea').value = deftext + text;
    };

    function generate(){
        var prefix = document.getElementById('main_textarea').value;
        var temp = document.getElementById('temp').value;
        var top_k = document.getElementById('top_k').value;
        var length = document.getElementById('length').value;
        
        var kernel = google.colab.kernel;
        var resultPromise = kernel.invokeFunction("ai_generate", [prefix,temp,top_k,length]);
        resultPromise.then(
            function(value) {
              add_text(desanitize(value.data["text/plain"]));
        });
    };
</script>
"""

HTML(input_form + javascript)